In [1]:
import os
import json

with open('config.json') as f:
    config = json.load(f)

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['TRANSFORMERS_CACHE'] = config['model_path']
os.environ['HF_HOME'] = config['model_path']

In [64]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [66]:
def loadModel(task, prompt, model_id, max_new_tokens):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id)

    pipe = pipeline(
        task=task,
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_new_tokens
    )

    hf = HuggingFacePipeline(pipeline=pipe)
    
    chain = prompt | hf | StrOutputParser()

    return chain

※ 한·영 모델의 경우, 당분간 openai 활용

1. 코드 생성 모델 

In [84]:
# prompt load
file = open('/opt/sample_prompt/prompt1/input/code.txt')
template_text = file.read()
prompt_template = PromptTemplate.from_template(template_text)

model_id = "codellama/CodeLlama-7b-Python-hf"
task = "text-generation"

codeGen = loadModel(
    task=task,
    prompt=prompt_template,
    model_id=model_id,
    max_new_tokens=512,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [85]:
question = "부산의 동래구 중 많은 매출을 올린 업종을 막대그래프로 그려줘"
codeGen_result = codeGen.invoke({'question':question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [87]:
print(codeGen_result)


"""

import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../input/store-sales-time-series-in-wild/wild_store_data.csv")

# 1. 데이터 프레임 생성
df = pd.read_csv("../input/store-sales-time-series-in-wild/wild_store_data.csv")

# 2. 데이터 프레임 내에서 필요한 데이터만 추출
df = df[df["region"] == "부산 동래구"]
df = df[["type_of_business", "amount"]]

# 3. 그래프 생성
df.groupby("type_of_business").sum().plot.bar()
plt.savefig("test_graph/test_graph.png")

# 4. 그래프 설명
with open("test_text/test_text.txt", "w") as f:
    f.write("This graph shows the amount of sales in Dongnae-gu, Busan.")



2. 영·한 번역 모델

In [91]:
# prompt load
file = open('/opt/sample_prompt/prompt1/input/eng-kr.txt')
template_text= file.read()
prompt_template = PromptTemplate.from_template(template_text)

model_id = "traintogpb/llama-2-en2ko-translator-7b-qlora-bf16-upscaled"
task = 'text-generation'

# load model
en2ko = loadModel(
    task=task, # task = 'translation' is not working
    prompt=prompt_template,
    model_id=model_id,
    max_new_tokens=200,
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [92]:
question = "Context의 내용을 번역해줘"
en2ko_result = en2ko.invoke({'question':question})

In [93]:
print(en2ko_result)


[Task]
이 막대 그래프는 제공된 신용 카드 데이터를 기반으로 부산 동래구의 매출 상위 업종을 보여줍니다. 각 막대는 다른 유형의 비즈니스를 나타내며 막대의 높이는 해당 산업에서 발생한 총 매출액(원화, KRW)을 나타냅니다. x 축은 비즈니스 유형을 나타내고 y 축은 총 매출액을 나타냅니다.[Task]
이 막대 그래프는 제공된 신용 카드 데이터를 기반으로 부산 동래구의 매출 상위 업종을 보여줍니다. 각 막대는 다른 유형의 비즈니스를 나타내며 막대의 높이는 해당 산업에서 발생한 총 매출액(원화, KRW)을 나타냅니다. x 축은 비즈니스 유형을 나타내고 y 축은 총 매출액을 나타냅니다.[Task]
이 막대 그래프는 제공된 신용 카드 데이터를 기반으로 부산 동래구의 매출 상위 업종을 보여줍니다. 각 막대는 다른 유형의 비즈니스를
